<h3>Modelo de clasificación de imagenes Pizza vs. Helado</h3>
<p>Aldo Tena García - A01275222</p>

<h4>Selección del dataset</h4>
<p>El dataset seleccionado para este modelo es "Pizza vs Ice Cream", fue descargado de la plataforma kaggle <ins>>(https://www.kaggle.com/datasets/hemendrasr/pizza-vs-ice-cream)</ins> el 14 de mayo del 2024.</p>
<p>La razón prinicipal para la selección de este dataset fue el objetivo de comparar 2 elementos diferentes usando el modelo a desarrollar, en este caso 2 alimentos diferentes, ademas de la estructura ya definida del set (con las carpetas train y test con sus respectivos tags) y la cantidad inicial de imagenes contenidas en el dataset (aproximadamente 500 imagenes de train y 50 de test).</p>